## Retriever And Chain With Langchain

In [1]:
## Data Ingestion
from langchain_community.document_loaders import TextLoader
loader=TextLoader("speech.txt")
docs=loader.load()
docs

[Document(page_content="Driver Status:\nWhen the driver's license (DL) expiration date has passed, we send a notification asking if you want to deactivate it or continue using it. If there is no response, we deactivate it automatically. Users can reactivate it, but they will receive a confirmation prompt.\nDriver App Login Criteria:\nWhen a driver enters their phone number, they will receive an OTP for validation. This will allow the driver to log in to all the companies where the phone number was used to create the driver account. The driver can easily switch between accounts.\nRequired fields to create a driver from web:\nTo create a driver, you need to provide the First Name and Phone Number as these are the required fields\nSearch Filter:\nYou can search for a driver by name, email, or phone number.       \n", metadata={'source': 'speech.txt'})]

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents=text_splitter.split_documents(docs)
documents

[Document(page_content="Driver Status:\nWhen the driver's license (DL) expiration date has passed, we send a notification asking if you want to deactivate it or continue using it. If there is no response, we deactivate it automatically. Users can reactivate it, but they will receive a confirmation prompt.\nDriver App Login Criteria:\nWhen a driver enters their phone number, they will receive an OTP for validation. This will allow the driver to log in to all the companies where the phone number was used to create the driver account. The driver can easily switch between accounts.\nRequired fields to create a driver from web:\nTo create a driver, you need to provide the First Name and Phone Number as these are the required fields\nSearch Filter:\nYou can search for a driver by name, email, or phone number.", metadata={'source': 'speech.txt'})]

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents,OllamaEmbeddings(model='llama3'))
db

In [8]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama3")
llm

Ollama(model='llama3')

In [18]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Suppose you're a customer support person and you need to resolve the customer's queries as best as you can. Don't reply with 'based on the provided context' as you're replying directly to the customer
<context>
{context}
</context>
Question: {input}""")

In [19]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [20]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x109a65b40>)

In [21]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [22]:
response=retrieval_chain.invoke({"input":"What is driver status"})
response

{'input': 'What is driver status',
 'context': [Document(page_content="Driver Status:\nWhen the driver's license (DL) expiration date has passed, we send a notification asking if you want to deactivate it or continue using it. If there is no response, we deactivate it automatically. Users can reactivate it, but they will receive a confirmation prompt.\nDriver App Login Criteria:\nWhen a driver enters their phone number, they will receive an OTP for validation. This will allow the driver to log in to all the companies where the phone number was used to create the driver account. The driver can easily switch between accounts.\nRequired fields to create a driver from web:\nTo create a driver, you need to provide the First Name and Phone Number as these are the required fields\nSearch Filter:\nYou can search for a driver by name, email, or phone number.", metadata={'source': 'speech.txt'})],
 'answer': "According to our system, when your driver's license (DL) expiration date passes, you'll

In [23]:
response['answer']

"According to our system, when your driver's license (DL) expiration date passes, you'll receive a notification asking if you'd like to deactivate it or continue using it. If we don't hear back from you, we'll automatically deactivate it. You can always reactivate it later, but you'll need to confirm that first."